In [1]:
!pip install img2vec_pytorch

In [2]:
from warnings import filterwarnings
filterwarnings(action='ignore', category=FutureWarning)

Let's use img2vec to build a dataset of tags, names, and vectors.

In [3]:
from img2vec_pytorch import Img2Vec
from PIL import Image
from arrow import now
from glob import glob
import pandas as pd
from os.path import basename

img2vec = Img2Vec(cuda=False, model='resnet-18', layer='default', layer_output_size=512)

GLOB_AI = '/kaggle/input/ai-generated-images-vs-real-images/AiArtData/AiArtData/*'
GLOB_REAL = '/kaggle/input/ai-generated-images-vs-real-images/RealArt/RealArt/*'

def get_from_glob(arg: str, tag: str) -> list:
    result = []
    for input_file in glob(pathname=arg):
        name = basename(input_file)
        try:
            with Image.open(fp=input_file, mode='r') as image:
                vector = img2vec.get_vec(image, tensor=True).numpy().reshape(512,)
                result.append(pd.Series(data=[tag, name, vector], index=['tag', 'name', 'value']))
        except RuntimeError:
            # we only have a few failures so we're just going to discard them
            pass
    return result

time_start = now()
ai = get_from_glob(arg=GLOB_AI, tag='ai')
print('done encoding the AI images in {}'.format(now() - time_start))
real = get_from_glob(arg=GLOB_REAL, tag='real')
df = pd.DataFrame(data=ai + real)
print('done in {}'.format(now() - time_start))
print(df.shape)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 86.3MB/s]


done encoding the AI images in 0:00:55.825918
done in 0:01:47.327806
(894, 3)


We have a DataFrame; what does it look like?

In [4]:
df.head()

,tag,name,value
0,ai,Various-AI-portraits-generated-by-Fotor.jpg,"[0.48017198, 0.8646128, 1.6611724, 0.915964, 0..."
1,ai,images77.jpg,"[1.4642816, 0.7189077, 1.1184237, 0.57213604, ..."
2,ai,1000_F_563719058_JXnzcPV4GRpWqmF5sqnqmbJ7ow3ca...,"[1.8749824, 0.036064498, 1.5152547, 1.4411104,..."
3,ai,MidJourney-content-policy-1024x576.jpg,"[0.9916628, 1.053051, 1.0515268, 1.0998507, 1...."
4,ai,an-ai-jungle-landscape-made-by-ai-landscape-ge...,"[0.9486984, 0.467991, 0.40774798, 0.9549215, 0..."


Let's use dimension reduction to visualize our data; if our vectors contain a signal that will may be useful for classification we should be able to see some evidence of it when we do dimension reduction. If our UMAP results do not show any clustering they do not.

In [5]:
from umap import UMAP
umap = UMAP(random_state=2024, verbose=True, n_jobs=1,)
model_df = pd.DataFrame(data=umap.fit_transform(X=df['value'].apply(pd.Series)), columns=['x', 'y'])
plot_df = pd.concat(objs=[df, model_df], axis=1)
print('done with UMAP in {}'.format(now() - time_start))

2024-02-20 16:25:31.013278: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 16:25:31.013445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 16:25:31.191434: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


UMAP(n_jobs=1, random_state=2024, verbose=True)
Tue Feb 20 16:25:46 2024 Construct fuzzy simplicial set
Tue Feb 20 16:25:47 2024 Finding Nearest Neighbors
Tue Feb 20 16:25:52 2024 Finished Nearest Neighbor Search
Tue Feb 20 16:25:56 2024 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Tue Feb 20 16:26:00 2024 Finished embedding
done with UMAP in 0:02:40.440153


In [6]:
from plotly.express import scatter
scatter(data_frame=plot_df, x='x', y='y', color='tag', hover_name='name', height=900)

Our image to vector vectors do seem to cluster somewhat according to whether they are real or generated. Let's split and use a model to do the classification.

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(df['value'].apply(pd.Series), df['tag'], test_size=0.25, random_state=2024)
model = LogisticRegression(max_iter=100000)
model.fit(X_train, y_train)

print('accuracy: {:5.2f} pct'.format(100 * accuracy_score(y_test, model.predict(X_test))))

accuracy: 75.00 pct


Of course our accuracy doesn't tell the whole story, so let's build a classification report.

In [8]:
from sklearn.metrics import classification_report
print(classification_report(y_true = y_test, y_pred = model.predict(X_test)))

              precision    recall  f1-score   support

          ai       0.80      0.78      0.79       135
        real       0.68      0.71      0.69        89

    accuracy                           0.75       224
   macro avg       0.74      0.74      0.74       224
weighted avg       0.75      0.75      0.75       224



Our classes are somewhat unbalanced; our model does a better job of finding generated images than real images, but this is not a terrible result for a first look.